1. BRITISH MUSEUM SEARCH

In [79]:
def british_museum_search(graph, start, goal):
    all_paths = []

    def explore(node, path):
        if node == goal:
            all_paths.append(path)
        for neighbor in graph.get(node, {}):
            if neighbor not in path:  
                explore(neighbor, path + [neighbor])

    explore(start, [start])

    if all_paths:
        optimal_path = min(all_paths, key=lambda x: sum(graph[x[i]][x[i + 1]] for i in range(len(x) - 1)))
        cost = sum(graph[optimal_path[i]][optimal_path[i + 1]] for i in range(len(optimal_path) - 1))
        return f"Optimal path: {' -> '.join(optimal_path)}, Cost: {cost}"
    return f"No path found from {start} to {goal}."


2. BREADTH FIRST SEARCH

In [80]:
def breadth_first_search(graph, start, goal):
    visited = set()
    queue = [(start, [start])] 

    while queue:
        node, path = queue.pop(0)
        if node == goal:
            cost = sum(graph[path[i]][path[i + 1]] for i in range(len(path) - 1))
            return f"BFS path: {' -> '.join(path)}, Cost: {cost}"

        visited.add(node)
        for neighbor in graph.get(node, {}):
            if neighbor not in visited:
                queue.append((neighbor, path + [neighbor]))

    return f"No path found from {start} to {goal}."


3. DEPTH FIRST SEARCH

In [81]:
def depth_first_search(graph, start, goal):
    visited = set()
    stack = [(start, [start])]

    while stack:
        node, path = stack.pop()
        if node == goal:
            cost = sum(graph[path[i]][path[i + 1]] for i in range(len(path) - 1))
            return f"DFS path: {' -> '.join(path)}, Cost: {cost}"

        visited.add(node)
        for neighbor in graph.get(node, {}):
            if neighbor not in visited:
                stack.append((neighbor, path + [neighbor]))

    return f"No path found from {start} to {goal}."


4. BEAM SEARCH

In [82]:
import heapq

def beam_search(graph, start, goal, beam_width):
    open_list = [(0, start, [start])]  
    closed_list = set()

    while open_list:
        open_list.sort()  
        open_list = open_list[:beam_width]  

        next_level = []
        for current_cost, current_node, path in open_list:
            if current_node == goal:
                return f"Beam Search path: {' -> '.join(path)}, Cost: {current_cost}"

            closed_list.add(current_node)

            for neighbor, cost in graph.get(current_node, {}).items():
                if neighbor in closed_list:
                    continue
                new_cost = current_cost + cost
                new_path = path + [neighbor]
                next_level.append((new_cost, neighbor, new_path))

        open_list = next_level

    return f"No path found from {start} to {goal}."



5. HILL CLIMBING

In [83]:
import random

def hill_climbing(graph, start, goal):
    heuristics = {node: random.randint(1, 10) for node in graph}
    
    heuristics[goal] = 0
    
    current_node = start
    path = [current_node]

    while current_node != goal:
        neighbors = graph.get(current_node, {})
        
        if not neighbors:
            return f"No path found from {start} to {goal}."

        next_node = min(neighbors, key=lambda neighbor: neighbors[neighbor] + heuristics[neighbor])

        if neighbors[next_node] + heuristics[next_node] >= neighbors.get(current_node, float('inf')) + heuristics.get(current_node, float('inf')):
            break
        
        path.append(next_node)
        current_node = next_node
    if current_node == goal:
        cost = sum(graph[path[i]][path[i + 1]] for i in range(len(path) - 1))
        return f"Hill Climbing Path: {' -> '.join(path)}, Cost: {cost}, Heuristics: {heuristics}"
    return f"No path found from {start} to {goal}."


6. ORACLE

In [84]:
def oracle_search(graph, start, goal):
    oracle_cost = float('inf')
    oracle_path = None
    
    def british_museum_search(node, current_path, current_cost):
        nonlocal oracle_cost, oracle_path
        
        if node == goal:
            if current_cost < oracle_cost:
                oracle_cost = current_cost
                oracle_path = list(current_path)
            return
        
        if current_cost >= oracle_cost:
            return
        
        for neighbor in graph.get(node, {}):
            if neighbor not in current_path:  
                current_path.append(neighbor)
                british_museum_search(neighbor, current_path, current_cost + graph[node][neighbor])
                current_path.pop()  

    british_museum_search(start, [start], 0)

    return oracle_path, oracle_cost 

7. BRANCH AND BOUND

In [85]:
import heapq

def branch_and_bound(graph, start, goal, oracle_cost):

    queue = []
    heapq.heappush(queue, (0, [start]))  
    visited = set()

    best_path = None
    best_cost = float('inf')

    while queue:
        cost, path = heapq.heappop(queue)
        current_node = path[-1]

        if current_node == goal:
            if cost < best_cost:
                best_cost = cost
                best_path = path
            continue

        if cost > oracle_cost:
            continue

        if current_node in visited:
            continue
        visited.add(current_node)
        for neighbor, weight in graph[current_node].items():
            if neighbor not in visited:
                new_cost = cost + weight
                heapq.heappush(queue, (new_cost, path + [neighbor]))

    if best_cost <= oracle_cost:
        oracle_comparison = f"Branch and Bound performed optimally or better with cost {best_cost} (Oracle cost: {oracle_cost})."
    else:
        oracle_comparison = f"Branch and Bound performed worse with cost {best_cost} (Oracle cost: {oracle_cost})."

    return best_path, best_cost, oracle_comparison

8. BRANCH AND BOUND + DEAD HORSE PRINCIPLE

In [86]:
def branch_and_bound_dead_horse(graph, start, goal, oracle_cost):
    stack = [(0, start, [start])]
    visited = set()  

    while stack:
        current_cost, current_node, path = stack.pop()

        if current_node == goal:
            return f"Branch and Bound (Dead Horse) path: {' -> '.join(path)}, Cost: {current_cost}"

        visited.add(current_node)

        for neighbor, cost in graph.get(current_node, {}).items():
            if neighbor in visited:
                continue  

            new_cost = current_cost + cost
            new_path = path + [neighbor]

            if new_cost > oracle_cost:
                continue

            stack.append((new_cost, neighbor, new_path))

    return f"No path found from {start} to {goal}."


9. BRANCH AND BOUND + HEURISTICS

In [87]:
import heapq

def branch_and_bound_heuristics(graph, start, goal, heuristic, oracle_cost):
    open_list = [(0, start, [start])]  
    closed_list = set()
    best_cost = float('inf')  

    while open_list:
        current_cost, current_node, path = heapq.heappop(open_list)

        if current_node == goal:
            if current_cost < best_cost:
                best_cost = current_cost
            return f"Branch and Bound (Heuristics) path: {' -> '.join(path)}, Cost: {best_cost}"

        closed_list.add(current_node)

        for neighbor, cost in graph.get(current_node, {}).items():
            if neighbor in closed_list:
                continue

            new_cost = current_cost + cost
            new_path = path + [neighbor]

            if new_cost >= oracle_cost:
                continue  

            if new_cost < best_cost:
                priority = new_cost + heuristic(neighbor, goal)
                heapq.heappush(open_list, (priority, neighbor, new_path))

    return f"No path found from {start} to {goal}."

10. A*

In [88]:
import heapq

def a_star(start, goal, graph): 

    def heuristic(node, goal):
        return abs(ord(goal) - ord(node))

    open_list = [(0, start)]  
    closed_list = set()  
    g_scores = {start: 0}  
    parents = {start: None}  

    oracle_path, oracle_cost = oracle_search(graph,start, goal)  

    while open_list:
        f_score, current = heapq.heappop(open_list)

        if current == goal:
            path = []
            while current is not None:
                path.append(current)
                current = parents[current]
            
            path = path[::-1]  
            cost = sum(graph[path[i]][path[i + 1]] for i in range(len(path) - 1))
            
            return f"Path: {' -> '.join(path)}, Cost: {cost}"

        closed_list.add(current)

        for neighbor, cost in graph[current].items():
            if neighbor in closed_list:
                continue

            tentative_g_score = g_scores[current] + cost

            if neighbor not in g_scores or tentative_g_score < g_scores[neighbor]:
                g_scores[neighbor] = tentative_g_score

                heuristic_cost = heuristic(neighbor, goal)
                if oracle_cost < heuristic_cost:
                    priority = tentative_g_score + oracle_cost
                else:
                    priority = tentative_g_score + heuristic_cost

                heapq.heappush(open_list, (priority, neighbor))
                parents[neighbor] = current

    return f"No path found from {start} to {goal}."


11. AO*

In [89]:
def ao_star(conditions, H, start, graph, goal):
    main_nodes = list(conditions.keys())
    main_nodes.reverse()
    least_cost = {}
    oracle_path, oracle_cost = oracle_search(graph, start, goal)

    print("Assumed Heuristic Values:")
    for node, value in H.items():
        print(f"{node}: {value}")
    
    for key in main_nodes:
        costs = {}
        
        if 'AND' in conditions[key]:
            AND_nodes = conditions[key]['AND']
            PathA = sum(H[node] for node in AND_nodes)  
            if PathA <= oracle_cost:  
                costs[' AND '.join(AND_nodes)] = PathA
                
        if 'OR' in conditions[key]:
            OR_nodes = conditions[key]['OR']
            PathB = min(H[node] for node in OR_nodes)  
            if PathB <= oracle_cost:  
                costs[' OR '.join(OR_nodes)] = PathB
        
        if costs:
            least_cost[key] = costs  
    
    def find_shortest_path(node):
        path = node
        total_cost = 0
        
        if node in least_cost.keys():
            min_cost = min(least_cost[node].values())  
            key = list(least_cost[node].keys())  
            values = list(least_cost[node].values())  
            index = values.index(min_cost)  
            next_nodes = key[index].split(' AND ' if 'AND' in key[index] else ' OR ')
            
            if len(next_nodes) == 1:
                next_node = next_nodes[0]
                subpath, subcost = find_shortest_path(next_node)  
                path += ' --> ' + subpath
                total_cost += subcost
            else:
                path += ' --> (' + key[index] + ') '
                subpaths = []
                for next_node in next_nodes:
                    subpath, subcost = find_shortest_path(next_node)
                    subpaths.append(subpath)
                    total_cost += subcost
                path += ' + '.join(subpaths)
        
        total_cost += H[node]  
        return path, total_cost
    
    path_result, path_cost = find_shortest_path(start)
    
    return f"AO* path: {path_result}, Cost of Path: {path_cost}"


12. BEST FIRST SEARCH

In [90]:
import heapq

def best_first_search(start, goal, heuristic, get_neighbors, graph):
    oracle_path,oracle_cost = oracle_search(graph, start, goal) 
    open_list = [(heuristic(start, goal), 0, start)] 
    closed_list = set() 
    parents = {start: None}
    best_path = None
    best_cost = float('inf')  
    
    while open_list:
        _, current_cost, current = heapq.heappop(open_list)

        if current == goal:
            path = []
            while current is not None:
                path.append(current)
                current = parents[current]
            path = path[::-1]  
            
            total_cost = sum(graph[path[i]][path[i + 1]] for i in range(len(path) - 1))
            
            if total_cost < oracle_cost:
                best_path = path
                best_cost = total_cost
            
            return f"Best First Search path: {' -> '.join(best_path)}, Cost: {best_cost}"  # Return valid path and cost

        closed_list.add(current)

        for neighbor in get_neighbors(current):
            if neighbor in closed_list:
                continue
            
            tentative_cost = current_cost + graph[current][neighbor]
            
            if tentative_cost < oracle_cost:  
                parents[neighbor] = current
                heapq.heappush(open_list, (heuristic(neighbor, goal), tentative_cost, neighbor))
    
    return f"No valid path found from {start} to {goal} within the oracle cost." 

In [91]:
import ipywidgets as widgets
from IPython.display import display
import heapq
import random

def run_algorithm(selected_algorithm, graph, start, goal):
    heuristic = lambda x, y: abs(ord(x) - ord(y))  
    oracle_path, oracle_cost = oracle_search(graph, start, goal)
    
    if selected_algorithm == 'BFS':
        return breadth_first_search(graph, start, goal)
    elif selected_algorithm == 'DFS':
        return depth_first_search(graph, start, goal)
    elif selected_algorithm == 'A*':
        return a_star(start, goal, graph)  
    elif selected_algorithm == 'Hill Climbing':
        return hill_climbing(graph, start, goal)
    elif selected_algorithm == 'Best First Search':
        return best_first_search(start, goal, heuristic, lambda node: graph[node].keys(), graph)
    elif selected_algorithm == 'AO*':
        H = {node: random.randint(1, 10) for node in graph.keys()}  
        conditions = {node: {} for node in graph.keys()}
        for node in graph:
            neighbors = list(graph[node].keys())
            if len(neighbors) > 1:
                conditions[node]['AND'] = neighbors 
            elif neighbors:
                conditions[node]['OR'] = neighbors  
        return ao_star(conditions, H, start, graph, goal)
    elif selected_algorithm == 'British Museum Search':
        return british_museum_search(graph, start, goal)
    elif selected_algorithm == 'Oracle Search':
        if oracle_path:
            return f"Oracle path: {' -> '.join(oracle_path)}, Cost: {oracle_cost}"
        else:
            return f"No path found from {start} to {goal}."
    elif selected_algorithm == 'Branch and Bound':
        return branch_and_bound(graph, start, goal, oracle_cost)
    elif selected_algorithm == 'Branch and Bound (Dead Horse)':
        return branch_and_bound_dead_horse(graph, start, goal, oracle_cost)
    elif selected_algorithm == 'Branch and Bound (Heuristics)':
        return branch_and_bound_heuristics(graph, start, goal, heuristic, oracle_cost)
    elif selected_algorithm == 'Beam Search':
        return beam_search(graph, start, goal, oracle_cost)
    else:
        return "Algorithm not implemented."

graph_text = widgets.Textarea(
    description='Graph (dict):',
    layout=widgets.Layout(width='500px', height='100px'))
start_text = widgets.Text(value='A', description='Start Node:')
goal_text = widgets.Text(value='D', description='Goal Node:')
algorithm_dropdown = widgets.Dropdown(
    options=['BFS', 'DFS', 'A*', 'Hill Climbing', 'Best First Search', 'Beam Search',
             'Branch and Bound', 'Branch and Bound (Dead Horse)', 'Branch and Bound (Heuristics)',
             'British Museum Search', 'Oracle Search', 'AO*'],
    description='Algorithm:')
search_button = widgets.Button(description="Search")
output = widgets.Output()

def run_search(button):
    try:
        graph = eval(graph_text.value)
        start = start_text.value
        goal = goal_text.value
        selected_algorithm = algorithm_dropdown.value
        result = run_algorithm(selected_algorithm, graph, start, goal)
    except Exception as e:
        result = f"Error: {e}"
    
    output.clear_output()
    with output:
        print(result)

search_button.on_click(run_search)

display(graph_text, start_text, goal_text, algorithm_dropdown, search_button, output)


Textarea(value='', description='Graph (dict):', layout=Layout(height='100px', width='500px'))

Text(value='A', description='Start Node:')

Text(value='D', description='Goal Node:')

Dropdown(description='Algorithm:', options=('BFS', 'DFS', 'A*', 'Hill Climbing', 'Best First Search', 'Beam Se…

Button(description='Search', style=ButtonStyle())

Output()